In [1]:
import sys
path = '/gpfs/commons/groups/gursoy_lab/mstoll/'
sys.path.append(path)
import numpy as np
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, classification_report
from codes.models.metrics import calculate_roc_auc



In [80]:
P0 = 0.5
P1 = 0.6
PX = 0.1

N = 100000
N_variables = 2


In [81]:
X = np.random.binomial(1, PX, N)
nb_X_1 = np.sum(X == 1)
Y = np.zeros(N)
Y_1 = np.random.binomial(1, P1, nb_X_1)
Y_0 = np.random.binomial(1, P0, N - nb_X_1)
Y[X==1] = Y_1
Y[X==0] = Y_0

In [76]:
X

In [66]:
corr = np.corrcoef(X, Y)[0, 1]
P0_calc = np.sum(Y[X==1]) / nb_X_1
P1_calc = np.sum(Y[X==0]) / (N - nb_X_1)
diff_p_calc = P0_calc - P1_calc

In [67]:
corr, diff_p_calc



In [68]:
## model de prediciton
X = X.reshape(N, 1)

In [69]:
features_names = ['x']
# Train a decision tree classifier
model = DecisionTreeClassifier()
model.fit(X, Y)

# Faire des prédictions sur l'ensemble de test
labels_pred_test = model.predict(X)
labels_pred_train = model.predict(X)
proba_test = model.predict_proba(X)[:, 1]
proba_train = model.predict_proba(X)[:, 1]


In [70]:
nb_positive_train = np.sum(labels_pred_train==0)
nb_negative_train = np.sum(labels_pred_train==1)
nb_positive_test = np.sum(labels_pred_test==0)
nb_negative_test = np.sum(labels_pred_test==1)

TP_test = np.sum((Y==0 )& (labels_pred_test == 0)) / nb_positive_test
FP_test = np.sum((Y==1 )& (labels_pred_test == 0)) / nb_positive_test
TN_test = np.sum((Y==1 )& (labels_pred_test == 1)) / nb_negative_test
FN_test = np.sum((Y== 0)& (labels_pred_test == 1)) / nb_negative_test

TP_train = np.sum((Y==0 )& (labels_pred_train == 0)) / nb_positive_train
FP_train = np.sum((Y==1 )& (labels_pred_train == 0)) / nb_positive_train
TN_train = np.sum((Y==1 )& (labels_pred_train == 1)) / nb_negative_train
FN_train = np.sum((Y== 0)& (labels_pred_train == 1)) / nb_negative_train

accuracy_train = accuracy_score(Y, labels_pred_train)
accuracy_test = accuracy_score(Y, labels_pred_test)

auc_test = calculate_roc_auc(Y, proba_test)
auc_train = calculate_roc_auc(Y, proba_train)

proba_avg_zero_test = 1- np.mean(proba_test[Y==0])
proba_avg_zero_train = 1- np.mean(proba_train[Y==0])
proba_avg_one_test = np.mean(proba_test[Y==1])
proba_avg_one_train = np.mean(proba_train[Y==1])

In [71]:
print(f'{TP_test=}') 
print(f'{FP_test=}')
print(f'{TN_test=}')
print(f'{FN_test=}')
print(f'{TP_train=}') 
print(f'{FP_train=}')
print(f'{TN_train=}')
print(f'{FN_train=}')
print(' ')
print(f'{auc_test=}')
print(f'{auc_train=}')
print(' ')
print(' ')
print(f'{accuracy_test=}')
print(f'{accuracy_train=}')
print(' ')
print(f'{proba_avg_zero_test=}')
print(f'{proba_avg_zero_train=}')
print(f'{proba_avg_one_test=}')
print(f'{proba_avg_one_train=}')

In [72]:
acc_predict = PX * max(P1, 1-P1) + (1 - PX) * max(P0, 1-P0)
acc_predict